# Lab 8 - Diffusion Models (DM) 
## Denoising Diffusion Probabilistic Models (DDPM) and Denoising Diffusion Implicit Models (DDIM)

This class of methods is based stochastic processes that iteratively refine a noisy sample to generate new data points. They are particularly effective for generating high-quality samples (images, text, audio ...) and have become a popular choice in generative modeling.


### Principles and key concepts

The key idea is to start with a simple noise distribution (latent) and **iteratively** denoise it to produce samples that resemble the training data.
Similarily to Normalizing Flows (NFs), diffusion models learn a transformation from a simple distribution (often Gaussian noise) to the data distribution, but they do so through a stochastic process (*Stochastic Differential Equation* or SDE) rather than a single deterministic mapping (neural decoder).
As a consequence, the latent representation of the data again lives in the same space as the data itself (like NFs), which is a key difference with GANs or VAEs that learn a latent space that is typically lower-dimensional than the data space.

The forward process (noising) is usually a Markov chain that gradually adds noise to the data which can be efficiently achieved, while the reverse process (denoising) is learned using a neural network that predicts the noise at each step.

### Modeling
The diffusion process is discretized as a time sequence where $t \in \llbracket 0,T \rrbracket$ typically divided into two main phases: 

- a stochastic forward diffusion process (from $t=0$ to $t=T$) that adds noise to a clean data point $x_0$ to produce a noisy sample $x_T$:
$$
    x_{t+1} = f_\theta(x_t, t) + \epsilon_t
$$
where $f_\theta$ is a (simple) parametric function of the current point $x_t$ that is linearly combined with the noise $\epsilon_t$ to be added at each step. Typically $\epsilon_t \sim \mathcal N(\mu_t, \Sigma_t)$ is a noise term sampled from a Gaussian distribution, such that the noise level increases with $t$. By the end of the process ($t=T$), the data is almost entirely noise, and $x_T$ is close to a Gaussian distribution that is easy to sample from.

- a reverse diffusion process (from $t=T$ to $t=0$) that denoises the noisy random samples $x_T$ to generate new data points:
$$    x_{t-1} = g_\theta(x_t, t) + \epsilon_t$$
where $g_\theta$ is a complex parametric function (a neural network) that learns to reverse the noise addition process
and $\epsilon_t$ is again typically a noise term sampled from a Gaussian distribution.


The training objective is to minimize the difference between the predicted noise and the actual noise added during the forward process, often using a mean squared error loss.

Once trained, the model can generate new samples by starting with a random noise vector and iteratively applying the learned reverse diffusion process to denoise it.

This approach has shown impressive results in generating high-quality images and has been widely adopted in various applications, including image synthesis, inpainting, and super-resolution.


### Architecture

Contrary to other generative models studied previously (GANs, NFs, VAEs) that are based on specific architectures (encoders, decoders, etc.), the architecture of diffusion models is solely based on a neural network that learns to transform a simple noise distribution into the data distribution.
This neural network $g_\theta$ can be any architecture that is suitable for the task (here a simple MLP), such as convolutional networks for image data (typically a U-Net) or recurrent networks for sequential data.
The only key requirements is that 

1. the input $x_t$ and output $y_t$ of the neural network $y_t = g_\theta(x_t, t)$ are of the same dimension as the data, and

2. the neural network $g_\theta(x_t, t)$ depends on the time step $t$ to allow the model to learn different transformations at each step of the diffusion process.

As the number of time steps is typically large (e.g. $T=1,000$), rather than learning a different neural network for each time step $t$, this last requirement is typically achieved by concatenating the time step $t$ to the input of the neural network, or by using a positional encoding of the time step (e.g., sinusoidal encoding) to inject the time information into the model.

---

References:
- Diffusion model : "Deep Unsupervised Learning using Nonequilibrium Thermodynamics", Sohl-Dickstein et al. (ICML, 2015)
- DDPM: "Denoising Diffusion Probabilistic Models" by Ho et al. (NeuRIPS 2020)
- DDIM: "Denoising Diffusion Implicit Models" by Song et al. (ICLR 2021)

julien rabin @ greyc.ensicaen.fr 2025


## Course Recap on DDPM (Denoising Diffusion Probabilistic Models)


### Forward diffusion process with DDPM

The Forward diffusion process is defined in DDPM as the iterative "variance-preserving" stochastic process:
$$
    x_t = \sqrt{1 - \beta_t} x_{t-1} + \sqrt{\beta_t} \epsilon
    \quad \text{ where } \epsilon \sim \mathcal N(0, I_d)
$$
where $\beta_t$ is a small positive constant that controls the amount of noise added at each step:
$$\boxed{\color{orange}\text{noise\ schedule}: \quad
    \beta_t =  \tfrac{t}{T}\beta_T + (1-\tfrac{t}{T}) \beta_0
    \quad \text{ with } T=1000, \beta_0 = 0.0001,  \beta_T = 0.02
}
$$
Note $I_d$ is the identity matrix for $d$-dimensional data $x_t \in \mathbb R^d$.
This means that the forward process is a fixed Markov chain that adds noise to the data:
$$
    q(x_t | x_{t-1}) = \mathcal N(x_t; \sqrt{1 - \beta_t} x_{t-1}, \beta_t I_d)
$$

Rather than iterating this stochastic process, a more efficient (and equivalent) way to define the forward diffusion process is to use a closed-form expression that directly computes the noisy sample $x_t$ from the original data point $x_0$:
$$    
    q(x_t | x_0) = \mathcal N(x_t; \sqrt{\bar \alpha_t} x_0, (1 - \bar \alpha_t) I_d)
$$
where $\alpha_t  = 1 - \beta_t$  and $\bar \alpha_t = \prod_{s=1}^{t} \alpha_s$ is the cumulative product of the noise coefficients up to time $t$.

$$\color{blue}
\boxed{\text{forward\ diffusion\ process}: \quad
    x_t = \sqrt{\bar \alpha_t} x_0 + \sqrt{1 - \bar \alpha_t} \epsilon
    \quad \text{ where } \epsilon \sim \mathcal N(0, I_d)
}
$$

### Reverse diffusion process with DDPM

The reverse diffusion process is learned by a neural network $g_\theta$.
In DDPM, a reparamtrization is used to directly train a denoising network $\varepsilon_\theta(x_t, t)$ that predicts the noise $\epsilon$ added at each step of the forward process.
The reverse process is defined as:
$$    p_\theta(x_{t-1} | x_t) = \mathcal N(x_{t-1}; \mu_\theta(x_t, t), \Sigma_\theta(x_t, t))$$
where the mean $\mu_\theta(x_t, t)$ and covariance $\Sigma_\theta(x_t, t)$ are predicted by the neural network $\varepsilon_\theta(x_t, t)$.
The mean is typically defined as:
$$    \mu_\theta(x_t, t) = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{\beta_t}{\sqrt{1 - \bar \alpha_t}} \varepsilon_\theta(x_t, t) \right)$$
and the covariance is usually set to a fixed value, such as $\Sigma_\theta(x_t, t) = \sigma_t^2 I_d$ where $\sigma_t^2$ is a small constant (in DDPM $\sigma_t^2 = \beta_t$, but other choices are possible).


$$\color{magenta}
\boxed{\text{backward\ diffusion\ process}: \quad
    x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{\beta_t}{\sqrt{1 - \bar \alpha_t}} \varepsilon_\theta(x_t, t) \right) + \sqrt{\beta_t} \epsilon
    \quad \text{ where } \epsilon \sim \mathcal N(0, I_d)
}
$$

### Training

The training of diffusion models such as DDPM is typically done by maximizing the likelihood of the data under the learned distribution:
$$
    \mathcal L(\theta) = \mathbb E_{x_0 \sim p_{data}} \left[ \log p_\theta(x_0) \right]
$$
where $p_\theta$ is the density of data points $x_0$ under the learned distribution which is obtained by iterating the diffusion process $x_T,x_{T-1}, ..., x_1, x_0$:
$$
    p_\theta(x_0) 
    = \int_{x_1, \ldots, x_T} p_\theta(x_0, x_1, \ldots, x_T) dx_1 \cdots dx_T
    %= \int_{x_1, \ldots, x_T} p_\theta(x_0|x_1) p_\theta(x_1|x_2) \cdots p_\theta(x_{T-1}|x_T) p(x_T) dx_1 \cdots dx_T
$$
where $p_\theta(x_0, x_1, \ldots, x_T)$ is the joint distribution of the data and the latent variables at each time step which can be factorized as:
$$    p_\theta(x_0, x_1, \ldots, x_T) = p(x_T) \prod_{t=1}^{T} p_\theta(x_{t-1} | x_t)$$
where $p(x_T)$ is the prior distribution of the latent variable $x_T$, that converges (large $T$) to a standard Gaussian distribution $\mathcal N(0,I_d)$.

As for VAEs, this likelihood is intractable to compute directly.
However, as demonstrated in DDPM (see details in course), it is possible to train the model by minimizing a reparametrized loss function that approximates the likelihood of the data (lower bound as in VAEs):
$$\boxed{\color{green}
    \min_\theta \mathbb{E}_{x_0, \varepsilon, t} \; \| \varepsilon - \varepsilon_\theta (x_t, t)  \|^2
}
$$
where $x_t$ is sampled from the forward diffusion process $q(x_t | x_0)$, that is using a random standard noise $\varepsilon$ added to $x_0$ to obtain $x_t$. Time step $t$ is a discrete random variable sampled uniformly from $\llbracket 0,T \rrbracket$.

### Inference with Denoising Diffusion Implicit Models (DDIM)

Once the model is trained, it can be used to generate new samples by starting from a random noise vector $x_T$ and iteratively applying the learned reverse diffusion process.

In DDIM, this reverse diffusion process is modified to allow for a more efficient sampling process that does not require the full Markov chain of steps.

Instead of sampling from the learned distribution at each step, DDIM uses a deterministic mapping by replacing the stochastic noise term with a **fixed noise term computed from the learned model**:
$$\boxed{\text{DDIM\ backward\ diffusion\ process}: \quad
\begin{aligned}
    x_{t-1} = \sqrt{\bar \alpha_{t-1}} {\color{red} \hat x_0} + \sqrt{1 - \bar \alpha_{t-1}}
\varepsilon_\theta (x_t,t)
\\
\text{ where }
    {\color{red} \hat x_0 = \tfrac{1}{\sqrt{\bar \alpha_t}}
\left(  x_{t} - \sqrt{1 - \bar \alpha_t} \varepsilon_\theta (x_t,t) \right) }
\end{aligned}
}
$$

### Implementation
We implement here in pytorch the a simple denoising model based on a MLP that is conditioned on the time step $t$ by concatenating it to the input $x_t$.


## Libraries import and useful functions

In [ ]:
import torch 
from torch import nn, Tensor

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

Data loader

In [ ]:
def sample_data(n: int, model = 'moons') -> Tensor:
    data_dim = 2  # Dimension of the data
    if model == 'moons':
        from sklearn.datasets import make_moons
        return Tensor(make_moons(n_samples=n, noise=0.05)[0])
    elif model == 'circles':
        from sklearn.datasets import make_circles
        return Tensor(make_circles(n_samples=n, noise=0.05, factor=0.5)[0])
    elif model == '2gmm':
        n_samples = n
        n = n//3
        X1 = torch.randn(n, data_dim) @ torch.tensor([[.05, -0.02],[-0.02, .4]]) + torch.tensor([[.0, 1.0]]).view(1, data_dim)
        X2 = torch.randn(n_samples - n, data_dim) @ torch.tensor([[.3, 0.05],[0.05, .05]]) + torch.tensor([[-1.0, 0.]]).view(1, data_dim)
        return torch.cat((X1,X2), dim=0)  # Concatenate the two sets of samples
    elif model == 'radial_gmm':
        K = 8
        n_samples = n
        samples_per_component = n_samples // K
        remainder = n_samples % K
        all_samples = []

        for k in range(K):
            radius = 3.
            # Angle for the mean on a circle
            theta = 2 * np.pi * k / K
            cs = np.cos(theta)
            sn = np.sin(theta)

            # Radial direction unit vector
            radial = torch.tensor([cs, sn], dtype=float)#.view(data_dim, 1)
            tangential = torch.tensor([-sn, cs], dtype=float)#.view(data_dim, 1)

            mean = radius * radial
            
            # Covariance matrix: elongated along radial direction
            cov = 0.3 * torch.outer(radial, radial).to(float) + 0.05 * torch.outer(tangential, tangential).to(float)

            # Generate samples
            n = samples_per_component + (remainder if k == K else 0)
            samples = torch.randn(n, data_dim, dtype=float) @ torch.linalg.cholesky(cov).to(float) + mean.to(float)
            all_samples.append(samples)

        return torch.cat(all_samples, dim=0).to(torch.float32)
    else:
        raise ValueError(f"Unknown model: {model}.")


## Exercice 1 : Implementing diffusion forward encoder and DDPM/DDIM reverse processes

Complete the following implementation of the `DDPM` class.
Recall that 
- the latent encoder in AE/VAE (or the discriminator in GAN) is replaced here by a forward diffusion process: here the `noising` method (described by the equation in the blue box above) is parametrized by the noise schedule $\beta_t$ (`self.beta`) (see above the boxed orange equation).
- the reverse diffusion process iterative `denoising` (see equations in magenta for DDPM and red boxes for DDIM) is using a neural network (here `self.net`) in the `forward` method to predict the noise added at each step.
- the neural network (here `self.net`) takes as input the noisy sample $x_t$ concatenated with a positional encoding of the time step $t$ to allow the model to learn different transformations at each step. This means that the input to the neural network is of dimension `data_dim + time_dim` where `time_dim` is the dimension of the positional encoding of the time step $t$ (here `data_dim=2` and `time_dim=1`) and the output is of dimension `data_dim=2`.
- (see exercise 3 at the end) during inference, the `DDIM_denoising` method can be used instead of the `denoising` method with larger time steps to increase sampling speed while preserving the sampling quality ...

In [ ]:
class DDPM(nn.Module):
    def __init__(self, dim: int = 2, h: int = 64, T : int = 1_000):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(???, ???),
        )
        
        self.T = T  # number of diffusion steps, t=0 is the initial clean position, t=T is the final gaussian position
        self.t = torch.linspace(1, T, T)  # diffusion steps
        beta0 = ...  # initial noise variance
        betaT = ...
        self.beta = beta0 + (betaT-beta0) * self.t/self.T  # linear schedule for the variance of the noise, i.e. x_t = \sqrt{1-\beta_t} x_0 + \sqrt{\beta_t} \varepsilon
        self.alpha = ... # alpha_t = 1 - beta_t
        self.alpha_bar = ... # \bar\alpha_t = \prod_{s=1}^t \alpha_s computed with torch.cumprod
        self.sigma = ... # noise standard deviation at each step : in DDPM, \sigma_t = \sqrt{\beta_t}
    
    def forward(self, t_idx: Tensor, x_t: Tensor) -> Tensor:
        # xt is of shape (batch_size, dim)
        # t_idx is of shape (batch_size, 1) : it is the index of the diffusion step, i.e. t in [0, T-1]
        
        # the NN \epsilon_\theta predicts the noise added to the clean position x_0 at time t
        # x_t = \sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t} \epsilon_\theta(t, x_t) 
        
        t_emb = (self.t[t_idx]/self.T).view(-1, 1) # simple embedding : normalize t to [0, 1]
        input = ... # concatenate x_t and t_emb along dimension 1 using torch.cat
        eps = self.net(input)
        return eps
    
    def noising(self, t_idx: Tensor, x_0: Tensor, eps: Tensor = None) -> Tensor: # forward diffusion step
        a = ... # use \bar\alpha_t at time t=t_idx 
        if eps is None:
            eps = ... # sample standard normal noise of the same shape as x_0
        
        x_t = ... # compute x_t using the closed form equation of the forward diffusion process (blue box above)
        return x_t
    
    def denoising(self, t_idx: int, x_t: Tensor, method : str = "DDPM") -> Tensor:
        # Note: t_idx is here the same single integer for the whole batch !
        t_idx = Tensor([t_idx]).repeat(x_t.shape[0], 1).long()  # ensure t_idx is a tensor of the right shape
        eps = ... # predicted noise to remove at time t_idx from x_t using self.forward
        
        if method.lower() == "ddpm": # stochastic denoising using magenta box equation
            
            hat_xt = ...
            
            return hat_xt
        
        elif method.lower() == "ddim" : # deterministic denoising
            
            hat_xt = ...
            
            return hat_xt
            
        else:
            raise ValueError("Unknown method: choose 'DDPM' or 'DDIM'")

    
    def DDIM_denoising(self, t_start: int, t_end: int, x_t: Tensor) -> Tensor:
        assert t_start > t_end, "t_end must be lower than t_start for backward DDIM denoising"
        t_start = Tensor([t_start]).repeat(x_t.shape[0], 1).long() 
        t_end   = Tensor([t_end  ]).repeat(x_t.shape[0], 1).long()
        
        eps = self.forward(t_idx=t_start, x_t=x_t)
        
        hat_xt = ...
        
        return hat_xt
        

In [ ]:
# Initialize the DDPM model
torch.manual_seed(42)  # For reproducibility
ddpm = ... # Initialize DDPM with appropriate parameters

# Generate synthetic data
n = 1_000
data_model = 'radial_gmm'  # Choose from 'moons', 'circles', '2gmm', 'radial_gmm'
X0 = ... # sample data using sample_data function

print("Forward diffusion steps (noising)")
n_steps = 10
fig, axes = plt.subplots(1, n_steps + 2, figsize=(30, 4), sharex=True, sharey=True)
t_idx = torch.linspace(0,ddpm.T-1,n_steps).long()

axes[0].scatter(X0.detach()[:, 0], X0.detach()[:, 1], s=10, color='red')
axes[0].set_title(f't = 0 (clean data)')
axes[0].set_xlim(-3.0, 3.0)
axes[0].set_ylim(-3.0, 3.0)

Xt = torch.randn_like(X0)  # prior
axes[n_steps+1].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10, color='green')
axes[n_steps+1].set_title(f't = \infty (gaussian prior)')
axes[n_steps+1].set_xlim(-3.0, 3.0)
axes[n_steps+1].set_ylim(-3.0, 3.0)
    
for i in range(n_steps):
    Xt = ddpm.noising(t_idx=t_idx[i].repeat(X0.shape[0], 1), x_0=X0, eps=None)
    axes[i + 1].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10)
    axes[i + 1].set_title(f't = {t_idx[i]+1}')
    

In [ ]:
print("Backward diffusion steps (denoising with *untrained* DDPM model)")
n_steps = ddpm.T
n_show = 10
idx_show = list(torch.linspace(0,ddpm.T-1,n_show).numpy().astype(int))  # time indices to show
fig, axes = plt.subplots(1, n_show + 2, figsize=(30, 4), sharex=True, sharey=True)
t_idx = torch.linspace(0,ddpm.T-1,n_steps).long()

axes[n_show+1].scatter(X0.detach()[:, 0], X0.detach()[:, 1], s=10, color='red')
axes[n_show+1].set_title(f'GT')
axes[n_show+1].set_xlim(-3.0, 3.0)
axes[n_show+1].set_ylim(-3.0, 3.0)
    
Xt = torch.randn_like(X0)  # start from a random gaussian noise
axes[0].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10, color='green')
axes[0].set_title(f'time t = {ddpm.T}')

%matplotlib inline
for i in range(n_steps-1,-1,-1): # reverse order from T to 0
    Xt = ddpm.denoising(t_idx=i, x_t=Xt)  # use the trained model to denoise
    if t_idx[i] in idx_show:
        j = n_show - idx_show.index(t_idx[i])
        axes[j].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10)
        axes[j].set_title(f't = {t_idx[i]}')
        plt.draw()


## Exercice 2 : Training and Inference with DDPM and DDIM

Complete the following cells to implement the training loop of the DDPM model on the 2D dataset chosen.

Recall that the optimisation problem (see the green box above) consists in minimizing the mean squared error between the predicted noise $\varepsilon_\theta (x_t, t)$ and the actual noise $\varepsilon$ added during the forward diffusion process to obtain $x_t$ from $x_0$.
The training impose to sample randomly 
- a batch of clean data points $x_0$ from the training dataset,
- a batch of noise vectors $\varepsilon$ from a standard normal distribution,
- a time step $t$ and the corresponding noisy samples $x_t$ from the forward diffusion process.

In [ ]:
ddpm = ... # reinitialize the model

optimizer = ... # use your favorite optimizer !
criterion = ... # see the green box equation

n_iter = 100
batch_size = 256

losses = []

pbar = tqdm(range(n_iter), desc="Training Denoising Network (DDPM)", unit="it")
        
for it in pbar :
    x_0 = ... # sample a batch of size batch_size from X0
    # randomly sample a point x_t for t in [1, T]
    t_idx = torch.randint(0, ddpm.T, (len(x_0), 1))  # t in [0, T-1]
    eps = torch.randn_like(x_0)  # random noise 
    x_t = ...  # apply the noising step
    
    optimizer.zero_grad()
    loss = criterion(ddpm(t_idx=t_idx, x_t=x_t), eps)
    loss.backward() # ||\epsilon_\theta(t, x_t) - \varepsilon||^2
    optimizer.step()
    
    losses.append(loss.item())
    pbar.set_postfix({'loss'  : f"{losses[-1]:.4f}"})

Plot the training loss

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.semilogy(losses, label='loss')
ax.set_xlabel('Iteration')
fig.suptitle('Training Loss')
ax.legend()
plt.show()


In [ ]:
methods = ["DDPM", "DDIM"]  # comparison between "DDPM" and "DDIM"
for method in methods:
    n_steps = ddpm.T
    n_show = 10
    idx_show = list(torch.linspace(0,ddpm.T-1,n_show).numpy().astype(int))  # time indices to show
    fig, axes = plt.subplots(1, n_show + 2, figsize=(30, 4), sharex=True, sharey=True)
    t_idx = torch.linspace(0,ddpm.T-1,n_steps).long()

    axes[n_show+1].scatter(X0.detach()[:, 0], X0.detach()[:, 1], s=10, color='red')
    axes[n_show+1].set_title(f'GT')
    axes[n_show+1].set_xlim(-3.0, 3.0)
    axes[n_show+1].set_ylim(-3.0, 3.0)
    
    torch.manual_seed(42)  # for reproducibility
    Xt = torch.randn_like(X0)  # start from a random gaussian noise
    axes[0].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10, color='green')
    axes[0].set_title(f'time t = {ddpm.T}')
    
    print(f"Backward diffusion steps (denoising with _trained_ {method} model) using {n_steps} steps, showing only {n_show}")
    for i in range(n_steps-1,-1,-1): # reverse order from T to 0
        Xt = ddpm.denoising(t_idx=i, x_t=Xt, method=method)  # use the trained model to denoise
        if t_idx[i] in idx_show:
            j = n_show - idx_show.index(t_idx[i])
            axes[j].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10)
            axes[j].set_title(f't = {t_idx[i]}')
            
    plt.show()

## (Bonus) Exercice 3: accelerated sampling using ODE solvers (DDIM)

While the denoising diffusion process can be seen as a discretization of a *stochastic differential equation (SDE)* (at each step some random noise is added),
the DDIM denoising process can be interpreted as a discretization of an **ordinary differential equation (ODE)**: from $t=T$ to $t=1$, solving
$$
    \frac{dx_t}{dt} = - f(x_t, t)
$$
where $f(x_t, t)$ is a function that depends on the current state $x_t$ and time $t$, here:
$$
    f(x_t, t) - x_t = \sqrt{\bar \alpha_t} {\color{red} \hat x_0(x_t,t)} +  \sqrt{1 - \bar \alpha_t}
\varepsilon_\theta (x_t,t)
$$

Different solvers can be used to solve this ODE, for instance:
- *Euler's method:* (already implemented in DDIM above), it consists in approximately solving 
$$
    \frac{dx_t}{dt} \approx \frac{x_t - x_{t-1}}{t - (t-1)} \Rightarrow x_{t-1} = x_t + f(x_t, t) 
$$ 
With large steps $\Delta t$ (>1 in our setting), we get :
$$
    x_{t-\Delta t} = x_t + f(x_t, t) \Delta t
$$

- *Heun's method:* improves upon Euler's method by taking an additional step to estimate the slope at the next time point, and then averaging the slopes to update the solution:
$$
    \begin{aligned}
    k_1 &= f(x_t, t) \\
    k_2 &= f(x_t + k_1 \Delta t, t + \  \Delta t) \\
    x_{t-1} &= x_t + \frac{1}{2}(k_1 + k_2) \Delta t
    \end{aligned}
$$
- *Runge-Kutta methods:* more advanced techniques that use multiple intermediate steps to achieve higher accuracy. The most common is the fourth-order Runge-Kutta method (RK4), which computes four slopes at each step and combines them to update the solution:
$$
    x_{t-1} = x_t + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4) \Delta t
$$
where 
$$ \begin{aligned}
    k_1 &= f(x_t, t) \\
    k_2 &= f(x_t + \frac{1}{2} k_1 \Delta t, t + \frac{1}{2} \Delta t) \\
    k_3 &= f(x_t + \frac{1}{2} k_2 \Delta t, t + \frac{1}{2} \Delta t) \\
    k_4 &= f(x_t + k_3 \Delta t, t + \Delta t)
\end{aligned} $$

In [ ]:
# acceleration with DDIM using large steps
n_steps = 20
n_show = 10
fig, axes = plt.subplots(1, n_show + 2, figsize=(30, 4), sharex=True, sharey=True)

tau_idx   = torch.linspace(0,ddpm.T-1,n_steps).long() # only n_steps time steps are used now !

axes[n_show+1].scatter(X0.detach()[:, 0], X0.detach()[:, 1], s=10, color='red')
axes[n_show+1].set_title(f'GT')
axes[n_show+1].set_xlim(-3.0, 3.0)
axes[n_show+1].set_ylim(-3.0, 3.0)

torch.manual_seed(42)  # for reproducibility
Xt = torch.randn_like(X0)  # start from a random gaussian noise
axes[0].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10, color='green')
axes[0].set_title(f'time t = {ddpm.T}')

print(f"Accelerated Backward diffusion steps with DDIM using {n_steps} steps, showing only {n_show}")
j = 0
for i in range(n_steps-1,0,-1): # reverse order from T to 0
    #print(f"Step {i}:{tau_idx[i]} / {n_steps-1} -> {i-1}:{tau_idx[i-1]}")
    Xt = ddpm.DDIM_denoising(t_start=tau_idx[i], t_end=tau_idx[i-1], x_t=Xt)
    if (ddpm.T - tau_idx[i]) > (j * ddpm.T) // (n_show-1):
        j += 1
        #print(f"Showing t = {tau_idx[i-1]} @ {j=}")
        axes[j].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10)
        axes[j].set_title(f't = {tau_idx[i-1]}')
# last step : predict the final denoised position at t=0
Xt = ddpm.DDIM_denoising(t_start=tau_idx[1], t_end=0, x_t=Xt)  
j = n_show
axes[j].scatter(Xt.detach()[:, 0], Xt.detach()[:, 1], s=10)
axes[j].set_title(f't = {tau_idx[0]}')  
    
plt.show()
